Calculate flux dependence on zenith and/or atmosphere
-----------------------------------------------------

This notebook creates *Figure 5* from the proceedings. Since there some more calculations involved it will take more time - up to tens of minutes depending on your hardware.

If running for the first time, a cache file will be created to store the interpolation parameters for the integrated density profiles. The second run will use the cache and run much quicker.

Note: When using the **native** Python interface to NRLMSISE-00, the depth integration is **extremely slow** (1-3 minutes). As soon as the lincese questions with the ctypes interface to the pure-C code are solved this will be not an issue.

In [1]:
#import the usual modules
%load_ext autoreload
%matplotlib inline
%autoreload 2
import os
from os.path import join
os.chdir('..')
import matplotlib.pyplot as plt
import numpy as np

from MCEq.core import MCEqRun
import CRFluxModels as pm
from mceq_config import mceq_config_without

Initialize the `mceq_run` object
--------------------------------

In [5]:
mceq_run = MCEqRun(

interaction_model='SIBYLL2.3_rc1',

primary_model=(pm.HillasGaisser2012, "H3a"),
#Do not provide any default values to avoid unnecessary initilizations
theta_deg=None,
density_model=None,

#Exclude the atmopheric setting from the configuration file
**mceq_config_without(['density_model'])

)


Hadrons:

"p-bar", "p", "n-bar", "n", "pi+", "pi-", "K0L", "K+", 
"K-"

Mixed:

"Lambda0-bar", "Lambda0", "Xi0-bar", "Xi0", "K0S", "Xi--bar", "Xi-", "Sigma--bar", 
"Sigma-", "Sigma+-bar", "Sigma+", "Omega--bar", "Omega-", "D+", "D-", "Ds+", 
"Ds-", "D0", "D0-bar", "XiC+-bar", "XiC+", "tau+", "tau-", "LambdaC+-bar", 
"LambdaC+", "XiC0-bar", "XiC0", "OmegaC0-bar", "OmegaC0", "pi0"

Resonances:

"eta", "Sigma0-bar", "Sigma0", "D*+", "D*-", "eta*", "jpsi", "phi", 
"SigmaC0-bar", "SigmaC0", "SigmaC+-bar", "SigmaC+", "SigmaC++-bar", "SigmaC++", "omega", "Xi*0-bar", 
"Xi*0", "Xi*--bar", "Xi*-", "SigmaC*0-bar", "SigmaC*0", "SigmaC*++-bar", "SigmaC*++", "SigmaC*+-bar", 
"SigmaC*+", "K*0", "K*0-bar", "K*+", "K*-", "Sigma*+-bar", "Sigma*+", "Sigma*0-bar", 
"Sigma*0", "Sigma*--bar", "Sigma*-", "etaC", "rho+", "rho-", "rho0", "Delta--bar", 
"Delta0-bar", "Delta+-bar", "Delta++-bar", "Delta++", "Delta+", "Delta0", "Delta-", "D*0", 
"D*0-bar", "XiC*0-bar", "XiC*+-bar", "XiC*+", "XiC*0", "Ds*+", "Ds*

Define what to calculate
------------------------

This example will calculate the flux for 5 different atmospheric profiles and 2 zenith angles. Zenith of 0 means vertical and 90 horizontal respectively. Note that the more inclide the shower trajectory is, the more integration steps *(read: calculation time)* are necessary. The total (conventional + prompt) fluxes will be stored in the `*_dict` dictionaries for plotting in the next step.

In [7]:
mup_dict, numu_dict, nue_dict = {}, {}, {}
for atm_tup in [(('CORSIKA', ('PL_SouthPole', 'January')), 'green'),
                (('CORSIKA', ('PL_SouthPole', 'August')), 'red'),
                (('MSIS00', ('SouthPole', 'January')), 'blue'),
                (('MSIS00', ('SouthPole', 'August')), 'cyan'),
                (('CORSIKA', ('BK_USStd', None)), 'black')]:


    mceq_run.set_density_model(atm_tup[0])
    for theta in [0., 90.]:
        
        mceq_run.set_theta_deg(theta)
        mceq_run.solve()

        mag = 3

        mup_dict[(theta, atm_tup)] = mceq_run.get_solution('total_mu+', mag) + \
                               mceq_run.get_solution('total_mu-', mag)

        numu_dict[(theta, atm_tup)] = mceq_run.get_solution('total_numu', mag) + \
                                mceq_run.get_solution('total_antinumu', mag)

        nue_dict[(theta, atm_tup)] = mceq_run.get_solution('total_nue', mag) + \
                               mceq_run.get_solution('total_antinue', mag)

MCEqRun::set_density_model():  CORSIKA ('PL_SouthPole', 'January')
MCEqRun::set_theta_deg():  0.0
CorsikaAtmosphere::calculate_density_spline(): Calculating spline of rho(X) for zenith 0.0 degrees.
.. took 2.28s
Average spline error: 2.00977967792e-16
MCEqRun::_calculate_integration_path():
MCEqRun::_forward_euler(): Solver will perform 4729 integration steps.

MCEqRun::_forward_euler(): time elapsed during integration: 3.7389998436 sec
MCEqRun::set_theta_deg():  90.0
CorsikaAtmosphere::calculate_density_spline(): Calculating spline of rho(X) for zenith 90.0 degrees.
.. took 1.99s
Average spline error: 1.98974763929e-16
MCEqRun::_calculate_integration_path():
MCEqRun::_forward_euler(): Solver will perform 50471 integration steps.

MCEqRun::_forward_euler(): time elapsed during integration: 49.5599999428 sec
MCEqRun::set_density_model():  CORSIKA ('PL_SouthPole', 'August')
MCEqRun::set_theta_deg():  0.0
CorsikaAtmosphere::calculate_density_spline(): Calculating spline of rho(X) for zeni

ImportError: No module named c_msis_interface

Plot with `matplotlib`
----------------------

In [ ]:
color_spectrum = ['b', 'r', 'g', 'orange', 'cyan', 'violet',
                  'brown', 'pink', 'yellow', 'lightblue']
titles = {('CORSIKA', 'PL_SouthPole', 'January'): 'CKA SP/Jan',
          ('CORSIKA', 'PL_SouthPole', 'August'): 'CKA SP/Aug',
          ('MSIS00', 'SouthPole', 'January'): 'MSIS00 SP/Jan',
          ('MSIS00', 'SouthPole', 'August'): 'MSIS00 SP/Aug',
          ('CORSIKA', 'BK_USStd', None): 'USStd'}
fig = plt.figure(figsize=(8.5, 3.5))
fig.set_tight_layout(dict(rect=[0.01, 0.01, 0.99, 0.97]))
e_grid = mceq_run.y.e_grid
compare_to = (('CORSIKA', 'BK_USStd', None), 'black')

for theta, atm_tup in mup_dict.keys():
    mup_comp = mup_dict[(theta, compare_to)]
    numu_comp = numu_dict[(theta, compare_to)]
    nue_comp = nue_dict[(theta, compare_to)]
    
    atm_config, atm_col = atm_tup
    if atm_config[1].startswith('BK'):
        continue    
    ls = '--'
    atm_title = '_nolabel_'
    if theta < 90.:
        ls='-'
        atm_title = titles[atm_config]

     
    plt.subplot(121)        
    plt.plot(e_grid, mup_dict[(theta, atm_tup)] / mup_comp, ls=ls, lw=1.5,
             color=atm_col, label=atm_title)
    plt.semilogx()
    plt.xlabel(r"$E_{\mu}$ [GeV]")
    plt.ylim([0.75, 1.1])

    plt.subplot(122)
    plt.plot(e_grid, numu_dict[(theta, atm_tup)] / numu_comp, ls=ls,  lw=1.5,
             color=atm_col, label=atm_title)
    plt.semilogx()
    plt.xlabel(r"$E_{\nu}$ [GeV]")


plt.subplot(121)
plt.title('Muons', fontsize=10)
plt.xlabel(r"$E_{\mu}$ [GeV]")
plt.ylabel(r"$\Phi_{\mu}($atm$)/\Phi_{\mu}($USStd)")
plt.xlim([50,1e9])
plt.ylim([0.75, 1.13])
plt.legend(loc='lower left', ncol=2, frameon=False, fontsize=10)

plt.subplot(122)
plt.title('Muon neutrinos', fontsize=10)
plt.xlabel(r"$E_{\nu_\mu}$ [GeV]")
plt.ylabel(r"$\Phi_{\nu_\mu}($atm$)/\Phi_{\nu_\mu}($USStd)")
plt.xlim([50,1e9])
plt.ylim([0.75, 1.13])
plt.legend(loc='lower left', ncol=2, frameon=False, fontsize=10)
plt.savefig('atm_flux.pdf')